In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install bs4

In [54]:
from bs4 import BeautifulSoup

In [55]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [56]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.80/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\euh27\.wdm\drivers\chromedriver\win32\98.0.4758.80]


In [57]:
driver.get("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/scheduleTreeCoursesIndex.faces")

In [ ]:
driver.find_element_by_css_selector('#dtree0 > div:nth-child(31) > a.node').click()

In [ ]:
driver.find_element_by_css_selector('#dtree16 > div:nth-child(7) > a.node').click()

In [ ]:
driver.find_element_by_css_selector('#stree278').click()

In [ ]:
driver.find_element_by_css_selector('#myForm > table:nth-child(6) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table > tbody > tr:nth-child(5) > td:nth-child(6) > a:nth-child(2) > table > tbody > tr > td:nth-child(2) > span').click()

In [58]:
content = driver.page_source
soup = BeautifulSoup(content,features='lxml')
rt= soup.find(id="myForm:timetable")
# rt

In [59]:
trs = rt.find_all('tr')
print(len(trs[1:]))

793


In [60]:
def press_details(index=1):
    details_selector = '#myForm\:timetable > tbody > tr:nth-child({}) > td:nth-child(8) > a'
    details_button = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,details_selector.format(index))))
    details_button.click()
# press_details(3)

In [61]:
def press_details_js(index=0):
    driver.execute_script(f"showToolTipDilaog(event,'{index}');")
# press_details()

In [ ]:
times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
if times:
for i in times:
    print(i.text)
len(times)

In [62]:
def get_times():
#     times = driver.find_element_by_id('sectionDiv').find_elements_by_tag_name('nobr')
    times = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.ID,'sectionDiv'))).find_elements_by_tag_name('nobr')
    if not times:
        return None
    n=len(times)//3
    return [times[i*3].text for i in range(n)], [times[1+i*3].text for i in range(n)] , [times[2+i*3].text for i in range(n)]
# tmp = {}
# tmp['days'], tmp['time'], tmp['location'] = get_times()
# print(tmp)

In [63]:
def get_final_info():
    return driver.find_element_by_id('examDayDiv').text,driver.find_element_by_id('examTimeDiv').text,driver.find_element_by_id('examDateDiv').text
# print(get_final_info())

In [64]:
def get_instructor():
    return driver.find_element_by_id('instructorDiv').text
# print(get_instructor())

In [65]:
def close():
    driver.find_element_by_css_selector('#dlg > div.pui-dialog-titlebar.ui-widget-header1.ui-helper-clearfix.ui-corner-top > a > span').click()

In [66]:
ctrs = trs[1:]
# print(ctrs)
len(ctrs)

793

In [67]:
res = []
x =0
for i in ctrs:
    tds = i.find_all('td');
    if 'مشروع' in tds[3].text or 'تدريب' in tds[3].text :
        close()
        x+=1
        continue
    tmp = {
        'cid': tds[0].text,
        'cname': tds[1].text,
        'section': tds[2].text,
        'activity': tds[3].text,
        'chours': tds[4].text,
        'gender': tds[5].text,
        'status': tds[6].text
    }
    press_details_js(x)
    driver.implicitly_wait(5)
    times = get_times()
    
    tmp['days'],tmp['time'],tmp['location']=(times) if times else ('','','')
    
    final_info = get_final_info()
    tmp['fexamday'],tmp['fexamtime'],tmp['fexamdate']=final_info
    
    tmp['instructor'] = get_instructor()
    driver.implicitly_wait(5)
    close()
    driver.implicitly_wait(5)
    res.append(tmp)
#     print(res)
    x+=1
#     if(x==5):
#         break;

In [ ]:
# y = [x.find_all('td') for x in ctrs]
# z = [i for i in y if i[3].text=='مشروع']
# for i in z:
#     if 'مشروع' not in i[3].text:
#         print(i)

In [68]:
len(res)

764

In [52]:
res[4]

{'رمز المقرر': '101 عرب',
 'اسم المقرر': 'المهارات اللغوية',
 'الشعبة': '3243',
 'النشاط': 'محاضرة',
 'الساعات': '2',
 'الجنس': 'الرياض- طلاب',
 'الحالة': 'مفتوحة',
 'days': ['1 3 5'],
 'time': ['  08:00 ص - 08:50 ص'],
 'location': ['  0140 04 1 A 097'],
 'fexamday': '2',
 'fexamtime': '01:00 م - 04:00 م',
 'fexamdate': '1443-10-29',
 'instructor': 'بندر عبدالله ناصر بن محسن'}

In [69]:
import csv
keys = res[0].keys()

with open('swe_courses.csv', 'w',encoding='utf-8-sig', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(res)